In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

In [ ]:
from ioUtils import getFile
data = getFile("/Volumes/Music/Discog/diagnostic/albumKnownIDs.p")

In [ ]:
data